In [1]:
import csv
import json
import requests
import pandas as pd

# Preparação

Os links do TSE exigem números bem específicos para o pleito presidencial e pleito estadual, bem como para cada cargo no pleito estadual. Estão já organizados em um dicionário:

In [2]:
cargo = ['Presidente', 'Governador', 'Senador', 'Deputado Federal', 'Deputado Estadual', 'Deputado Distrital']
pleito = ['544', '546', '546', '546', '546', '546']
codigo = ['1', '3', '5', '6', '7', '8']

cargos = {}
for item in range(len(cargo)):
    cargos[cargo[item]] = codigo[item], pleito[item]

cargos

{'Presidente': ('1', '544'),
 'Governador': ('3', '546'),
 'Senador': ('5', '546'),
 'Deputado Federal': ('6', '546'),
 'Deputado Estadual': ('7', '546'),
 'Deputado Distrital': ('8', '546')}

Para cada local, faço a classificação abaixo com o tipo de pleito e cargos a serem buscados.

In [3]:
UFs = ["BR", "ZZ", "DF", "AC", "AL", "AP", "AM", "BA", "CE", "ES", "GO", "MA", "MT", "MS", "MG", "PR", "PB", "PA", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SE", "SP", "TO"]
# ZZ é a forma que o TSE conta os votos no exterior

ufs = []
aux = ['presidente', 'presidente', 'df']
for item in range(len(UFs)-3):   # Olha, talvez tivesse até sido mais fácil copiar e colar,
    aux.append('estado')         # mas certamente seria mais tedioso.
for item in range(len(UFs)):
    ufs.append([UFs[item], aux[item]])
    
ufs[0:5]

[['BR', 'presidente'],
 ['ZZ', 'presidente'],
 ['DF', 'df'],
 ['AC', 'estado'],
 ['AL', 'estado']]

In [4]:
cargo_local = {'presidente':['Presidente'],
              'df':['Presidente', 'Governador', 'Senador', 'Deputado Federal', 'Deputado Distrital'],
              'estado':['Presidente', 'Governador', 'Senador', 'Deputado Federal', 'Deputado Estadual']}
# vale lembrar que cada UF tem seu pleito

A lista de municipios é dada.

In [5]:
municipios = pd.read_csv(r'Eleições\municipios_brasileiros_tse.csv') # minha pasta padrão para este projeto chama Eleições
municipios

,codigo_tse,uf,nome_municipio,capital,codigo_ibge
0,1120,AC,ACRELÂNDIA,0,1200013
1,1570,AC,ASSIS BRASIL,0,1200054
2,1058,AC,BRASILÉIA,0,1200104
3,1007,AC,BUJARI,0,1200138
4,1015,AC,CAPIXABA,0,1200179
...,...,...,...,...,...
5565,96199,TO,TOCANTÍNIA,0,1721109
5566,73458,TO,TUPIRAMA,0,1721257
5567,73237,TO,TUPIRATINS,0,1721307
5568,96652,TO,WANDERLÂNDIA,0,1722081


# Definição de funções:
Abaixo defino as funções para os três tipos de arquivo que o TSE disponibiliza (normal, simplificado e por cidade), e logo depois um breve teste.

In [3]:
def links_n(CARGO = '1', PLEITO = '544', UF = 'BR'):
    url_dados = ("https://resultados.tse.jus.br/oficial/ele2022/" + str(PLEITO) + "/dados-simplificados/" + 
                 UF.lower() + "/" + UF.lower() + "-c000" + str(CARGO) + "-e000" + str(PLEITO) + "-r.json")
    return url_dados
        
def links_s(CARGO = '1', PLEITO = '544', UF = 'BR'):
    url_dados_simplificados = ("https://resultados.tse.jus.br/oficial/ele2022/" + str(PLEITO) + "/dados/" + UF.lower()
                + "/" + UF.lower() + "-c000" + str(CARGO) + "-e000" + str(PLEITO) + "-v.json")
    return url_dados_simplificados

def links_c(CARGO = '1', PLEITO = '544', UF = 'MG', CIDADE ='41238'):
    url_dados_cidade = ("https://resultados.tse.jus.br/oficial/ele2022/" + str(PLEITO) + "/dados/" + UF.lower()
                + "/" + UF.lower() + '{:0>5}'.format(CIDADE) + "-c000" + str(CARGO) + "-e000" + str(PLEITO) + "-v.json")
    return url_dados_cidade

In [4]:
a, b, c = links_n(), links_s(), links_c()
print(a, "\n\n", b, "\n\n", c, sep='')

https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/br/br-c0001-e000544-r.json

https://resultados.tse.jus.br/oficial/ele2022/544/dados/br/br-c0001-e000544-v.json

https://resultados.tse.jus.br/oficial/ele2022/544/dados/mg/mg41238-c0001-e000544-v.json


# Geração de Arquivo

## Arquivos normais
Lista de links possíveis

In [19]:
normal_links = []

for uf in ufs:
    for cargo_uf in cargo_local[uf[1]]:
        filename = uf[0] + '_' + cargo_uf + '.json' 
        normal_links.append([uf[0], cargo_uf, links_n(*cargos[cargo_uf], uf[0]), filename])
normal_links

[['BR',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/br/br-c0001-e000544-r.json',
  'BR_Presidente.json'],
 ['ZZ',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/zz/zz-c0001-e000544-r.json',
  'ZZ_Presidente.json'],
 ['DF',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados-simplificados/df/df-c0001-e000544-r.json',
  'DF_Presidente.json'],
 ['DF',
  'Governador',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/df/df-c0003-e000546-r.json',
  'DF_Governador.json'],
 ['DF',
  'Senador',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/df/df-c0005-e000546-r.json',
  'DF_Senador.json'],
 ['DF',
  'Deputado Federal',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados-simplificados/df/df-c0006-e000546-r.json',
  'DF_Deputado Federal.json'],
 ['DF',
  'Deputado Distrital',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dad

Download de arquivos

In [ ]:
start = 'Eleições\\normal_json\\' # insira aqui a pasta que você preferir
for line in normal_links:
    r = requests.get(line[2]).json()
    filename = start + line[3]
    with open(filename, 'w') as openfile:
        json.dump(r, openfile)

Lista em csv para arquivamento

In [30]:
with open('Eleições\\normal_links.csv', 'w') as f: # insira aqui a pasta que você preferir
    csv.writer(f).writerow(['UF', 'Cargo', 'Link', 'file'])
    csv.writer(f).writerows(normal_links)

## Arquivos simplificados

In [20]:
simple_links = []

for uf in ufs:
    for cargo_uf in cargo_local[uf[1]]:
        filename = uf[0] + '_' + cargo_uf + '_s.json' 
        simple_links.append([uf[0], cargo_uf, links_s(*cargos[cargo_uf], uf[0]), filename])
simple_links

[['BR',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados/br/br-c0001-e000544-v.json',
  'BR_Presidente_s.json'],
 ['ZZ',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados/zz/zz-c0001-e000544-v.json',
  'ZZ_Presidente_s.json'],
 ['DF',
  'Presidente',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados/df/df-c0001-e000544-v.json',
  'DF_Presidente_s.json'],
 ['DF',
  'Governador',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/df/df-c0003-e000546-v.json',
  'DF_Governador_s.json'],
 ['DF',
  'Senador',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/df/df-c0005-e000546-v.json',
  'DF_Senador_s.json'],
 ['DF',
  'Deputado Federal',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/df/df-c0006-e000546-v.json',
  'DF_Deputado Federal_s.json'],
 ['DF',
  'Deputado Distrital',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/df/df-c0008-e000546-v.json',
  'DF_Deputado Distrital_s.json'],
 ['AC

Download de arquivos

In [24]:
start = 'Eleições\\simple_json\\' # insira aqui a pasta que você preferir
for line in simple_links[87:]:
    r = requests.get(line[2]).json()
    filename = start + line[3]
    with open(filename, 'w') as openfile:
        json.dump(r, openfile)

Lista em csv para arquivamento

In [ ]:
with open('Eleições\\simple_links.csv', 'w') as f: # insira aqui a pasta que você preferir
    csv.writer(f).writerow(['UF', 'Cargo', 'Link', 'file'])
    csv.writer(f).writerows(simple_links)

## Arquivos por cidade

In [25]:
city_links = []

for row in municipios.index:
    cidade, uf, nome = municipios[['codigo_tse', 'uf', 'nome_municipio']].loc[row]
    localx = ufs[UFs.index(uf)][1]
    for cargo_uf in cargo_local[localx]:
        filename = uf + '_' + cargo_uf + '_' + '{:0>5}'.format(cidade) + '_' + nome + '_s.json' 
        city_links.append([uf, cargo_uf, '{:0>5}'.format(cidade), nome, links_c(*cargos[cargo_uf], uf, cidade), filename])

city_links

[['AC',
  'Presidente',
  '01120',
  'ACRELÂNDIA',
  'https://resultados.tse.jus.br/oficial/ele2022/544/dados/ac/ac01120-c0001-e000544-v.json',
  'AC_Presidente_01120_ACRELÂNDIA_s.json'],
 ['AC',
  'Governador',
  '01120',
  'ACRELÂNDIA',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/ac/ac01120-c0003-e000546-v.json',
  'AC_Governador_01120_ACRELÂNDIA_s.json'],
 ['AC',
  'Senador',
  '01120',
  'ACRELÂNDIA',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/ac/ac01120-c0005-e000546-v.json',
  'AC_Senador_01120_ACRELÂNDIA_s.json'],
 ['AC',
  'Deputado Federal',
  '01120',
  'ACRELÂNDIA',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/ac/ac01120-c0006-e000546-v.json',
  'AC_Deputado Federal_01120_ACRELÂNDIA_s.json'],
 ['AC',
  'Deputado Estadual',
  '01120',
  'ACRELÂNDIA',
  'https://resultados.tse.jus.br/oficial/ele2022/546/dados/ac/ac01120-c0007-e000546-v.json',
  'AC_Deputado Estadual_01120_ACRELÂNDIA_s.json'],
 ['AC',
  'Presidente',
  '01570',
  'AS

Download de arquivos

In [31]:
start = 'Eleições\\city_json\\' # insira aqui a pasta que você preferir
for line in city_links:
    r = requests.get(line[4]).json()
    filename = start + line[5] 
    with open(filename, 'w') as openfile:
        json.dump(r, openfile)

Lista em csv para arquivamento

In [ ]:
with open('Eleições\\city_links.csv', 'w') as f: # insira aqui a pasta que você preferir
    csv.writer(f).writerow(['UF', 'Cargo', 'cod_tse', 'Cidade', 'Link', 'file'])
    csv.writer(f).writerows(city_links)